### 1) Load the data


In [3]:
import matplotlib.pyplot as plt
import pandas as pd


column_types = {
    "departamento": "category",
    "municipio": "category",
    "sexo": "category",
    "año_registrado": "category",
    "edad": "int64",
    "periodo": "category",
    "etnia": "category",
    "escolaridad": "category",
    "ocupacion": "category",
    "causa": "category",
    "asistencia": "category",
    "lugar": "category",
}

defunciones = pd.read_csv("defunciones.csv", dtype=column_types)
defunciones.head()

/var/folders/jq/zj1d7dc15dd57tj80xs_lvxr0000gn/T/ipykernel_68314/2915034293.py:20: DtypeWarning: Columns (18,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  defunciones = pd.read_csv("defunciones.csv", dtype=column_types)


,Depreg,Mupreg,Mesreg,Añoreg,Depocu,Mupocu,Areag,Sexo,Diaocu,Mesocu,...,Dredif,Mredif,Caudef,Asist,Ocur,Cerdef,Puedif,Ciuodif,caudef.descrip,Añoocu
0,17.0,1703,6.0,2012.0,17.0,1703,1.0,1.0,25.0,6.0,...,23.0,2300,M329,5.0,6.0,1.0,NaN,NaN,NaN,NaN
1,1.0,101,6.0,2012.0,1.0,101,1.0,2.0,14.0,6.0,...,23.0,2300,B208,1.0,1.0,1.0,NaN,NaN,NaN,NaN
2,1.0,101,11.0,2012.0,1.0,101,1.0,2.0,22.0,11.0,...,23.0,2300,Q441,1.0,1.0,1.0,NaN,NaN,NaN,NaN
3,1.0,115,3.0,2012.0,1.0,115,1.0,1.0,19.0,3.0,...,23.0,2300,X954,5.0,5.0,1.0,NaN,NaN,NaN,NaN
4,1.0,101,11.0,2012.0,1.0,101,1.0,2.0,20.0,11.0,...,23.0,2300,C819,1.0,1.0,1.0,NaN,NaN,NaN,NaN


### 2. Preprocessing

1. Simplify ICD10 Code
2. Use age_groups instead of age
3. Handle Missing Values

In [8]:
# ================================================================
# Script unificado de limpieza y transformación usando CIE.csv
# (Asumiendo que CIE.csv tiene columnas "Codigo" y "Descripcion")
# ================================================================

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Carga y tipificación inicial
df = pd.read_csv("defunciones.csv")
col_types = {col: str for col in df.columns}
col_types["Añoreg"] = int
col_types["Edadif"] = int

# 2. Selección y renombre de columnas
defunciones = pd.read_csv("defunciones.csv", dtype=col_types)
defunciones = defunciones[
    [
        "Depocu", "Mupocu", "Sexo", "Añoreg", "Edadif",
        "Perdif", "Puedif", "Escodif", "Ciuodif", "Caudef",
        "Asist", "Ocur"
    ]
]
defunciones.rename(columns={
    "Depocu": "departamento",
    "Mupocu": "municipio",
    "Sexo": "sexo",
    "Añoreg": "año_registrado",
    "Edadif": "edad",
    "Perdif": "periodo",
    "Puedif": "etnia",
    "Escodif": "escolaridad",
    "Ciuodif": "ocupacion",
    "Caudef": "causa",
    "Asist": "asistencia",
    "Ocur": "lugar"
}, inplace=True)
defunciones.sort_values(by="año_registrado", inplace=True)
defunciones.to_csv("defunciones_clean.csv", index=False)

# 3. Conversión de columnas a categoría (excepto 'año_registrado' y 'edad')
for c in defunciones.columns:
    if c not in ["año_registrado", "edad"]:
        defunciones[c] = defunciones[c].astype(str).str.strip().astype("category")

# 4. Recarga del CSV limpio con tipos
column_types = {
    "departamento": "category",
    "municipio": "category",
    "sexo": "category",
    "año_registrado": "category",
    "edad": "int64",
    "periodo": "category",
    "etnia": "category",
    "escolaridad": "category",
    "ocupacion": "category",
    "causa": "category",
    "asistencia": "category",
    "lugar": "category",
}
defunciones = pd.read_csv("defunciones_clean.csv", dtype=column_types)
print("Vista previa de defunciones_clean:")
display(defunciones.head())

# 5. Lectura de CIE.csv (asumiendo columnas "Codigo" y "Descripcion")
def load_codes_csv(file_path):
    df_cie = pd.read_csv(file_path)
    codes_dict = {}
    # Observa que uso row["codigo"] y row["descripcion"] en minúsculas
    for _, row in df_cie.iterrows():
        codigo = str(row["codigo"]).strip()
        descripcion = str(row["descripcion"]).strip()
        codes_dict[codigo] = descripcion
    return codes_dict

def simplify_icd10(code, code_map):
    if pd.isna(code):
        return "Unknown"
    code_clean = code.strip()
    # Si se desea coincidencia exacta:
    if code_clean in code_map:
        return code_map[code_clean]
    # Si no encuentra exacto, se podria intentar usar el prefijo:
    prefix = code_clean.split(".")[0]
    for c in code_map:
        if c.startswith(prefix):
            return code_map[c]
    return "Other"

def split(defunciones_df, code_map):
    """
    - Elimina filas con NA
    - Crea 'causa_simplificada' y descarta "Other"
    - Reemplaza ocupación desconocida
    - Crea columna 'age_group' y elimina 'edad'
    """
    data = defunciones_df.dropna().copy()
    data["causa_simplificada"] = data["causa"].apply(lambda x: simplify_icd10(x, code_map))
    data = data[data["causa_simplificada"] != "Other"]
    data.drop(columns=["causa"], inplace=True)
    data.rename(columns={"causa_simplificada": "causa"}, inplace=True)

    mode = data["ocupacion"].mode()[0]
    data["ocupacion"] = data["ocupacion"].replace(["NEOG", "IGNORADO"], mode)

    data["age_group"] = pd.cut(
        data["edad"],
        bins=[0, 18, 35, 50, 65, float("inf")],
        labels=["0-18","19-35","36-50","51-65","65+"],
    )
    data.drop(columns=["edad"], inplace=True)
    data.to_csv("defunciones_simplified.csv", index=False)
    return data

# 6. Aplicación final
code_map = load_codes_csv("CIE.csv")  # Ajusta el path y nombres si es necesario
causes = split(defunciones, code_map)
print("Vista previa de defunciones_simplified:")
display(causes.head())

/var/folders/jq/zj1d7dc15dd57tj80xs_lvxr0000gn/T/ipykernel_68314/1440262573.py:12: DtypeWarning: Columns (18,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("defunciones.csv")


Vista previa de defunciones_clean:


,departamento,municipio,sexo,año_registrado,edad,periodo,etnia,escolaridad,ocupacion,causa,asistencia,lugar
0,17.0,1703,1.0,2012,28,3.0,NaN,1.0,NaN,M329,5.0,6.0
1,1.0,0101,2.0,2012,88,3.0,NaN,2.0,NaN,E142,1.0,6.0
2,1.0,0101,2.0,2012,74,3.0,NaN,2.0,NaN,E039,1.0,1.0
3,1.0,0101,2.0,2012,43,3.0,NaN,2.0,NaN,E149,1.0,6.0
4,1.0,0101,2.0,2012,88,3.0,NaN,2.0,NaN,E119,1.0,6.0


/var/folders/jq/zj1d7dc15dd57tj80xs_lvxr0000gn/T/ipykernel_68314/1440262573.py:106: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data["ocupacion"] = data["ocupacion"].replace(["NEOG", "IGNORADO"], mode)


Vista previa de defunciones_simplified:


,departamento,municipio,sexo,año_registrado,periodo,etnia,escolaridad,ocupacion,asistencia,lugar,causa,age_group
70905,14.0,1415,1.0,2013,3.0,1.0,1.0,61,5.0,6.0,"Úlcera gástrica, no especificada como aguda ni...",65+
70906,14.0,1411,1.0,2013,3.0,1.0,1.0,61,5.0,6.0,"Úlcera gástrica, no especificada como aguda ni...",65+
70907,16.0,1601,1.0,2013,3.0,1.0,9.0,92,5.0,6.0,"Úlcera gástrica, no especificada como aguda ni...",65+
70908,1.0,0116,1.0,2013,3.0,9.0,1.0,61,5.0,6.0,"Úlcera gástrica, no especificada como aguda ni...",65+
70909,12.0,1219,1.0,2013,3.0,4.0,1.0,92,5.0,6.0,"Úlcera gástrica, no especificada como aguda ni...",65+
